In [1]:
from jupyter_dash import JupyterDash
from dash import dcc, html
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
from dash.dependencies import Input, Output
from sklearn.tree import DecisionTreeRegressor, plot_tree
import matplotlib.pyplot as plt
import io
import numpy as np
import base64

/Users/hellenqian/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/hellenqian/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [2]:
colors = {
    'background': '#F5F5F5',
    'text': '#4A4A4A',
    'forest_green': '#2C5F2D',
    'ocean_blue': '#1D4E89',
    'light_gray_green': '#A4C3A3',
    'light_blue_gray': '#8FAFC0',
    'orange': '#F28C28',
    'lemon_yellow': '#FFD700',
}

df = pd.read_csv('./data.csv')
# 数据预处理：移除空值
df = df.dropna(subset=['Sector', 'Total ESG Risk score', 'Environment Risk Score', 'Governance Risk Score', 'Social Risk Score'])

# 创建 Dash 应用
app = JupyterDash(__name__)

/Users/hellenqian/opt/anaconda3/lib/python3.9/site-packages/dash/dash.py:585: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



In [3]:
app.layout = html.Div(style={'backgroundColor': colors['background']}, children=[
    html.H1(children='ESG Score Analysis', style={'textAlign': 'center', 'color': colors['text']}),

    # Center the dropdown with a fixed width
    html.Div(
        dcc.Dropdown(
            id='sector-dropdown',
            options=[{'label': sector, 'value': sector} for sector in df['Sector'].unique()],
            value=df['Sector'].unique()[0],
            style={
                'color': colors['text'],
                'width': '300px',  # Set the width of the dropdown
                'margin': '0 auto'  # Center the dropdown
            },
        ),
        style={'textAlign': 'center'}  # Center the container
    ),

    dcc.Graph(id='esg-bar-chart'),
    dcc.Graph(id='esg-3d-scatter-chart'),
    dcc.Graph(id='esg-extreme-value-chart'),

    html.Div(children=[
        dcc.Link('Back to Home', href='/'),
        html.Br(),
        dcc.Link('View Other Analyses', href='/other-analysis')
    ], style={'textAlign': 'center', 'color': colors['text']}),
]) 

In [4]:

# Callback for the bar chart
@app.callback(
    Output('esg-bar-chart', 'figure'),
    Input('sector-dropdown', 'value')
)
def update_bar_graph(selected_sector):
    # Filter data based on selected sector
    filtered_df = df[df['Sector'] == selected_sector]

    # Create stacked bar chart
    fig = go.Figure()

    # Add environmental score
    fig.add_trace(go.Bar(
        x=filtered_df['Name'],
        y=filtered_df['Environment Risk Score'],
        name='Environment Score',
        marker_color=colors['forest_green'],
    ))

    # Add governance score
    fig.add_trace(go.Bar(
        x=filtered_df['Name'],
        y=filtered_df['Governance Risk Score'],
        name='Governance Score',
        marker_color=colors['orange'],
    ))

    # Add social score
    fig.add_trace(go.Bar(
        x=filtered_df['Name'],
        y=filtered_df['Social Risk Score'],
        name='Social Score',
        marker_color=colors['ocean_blue'],
    ))

    # Update layout
    fig.update_layout(
        title=f'ESG Scores for {selected_sector} Sector',
        xaxis_title='Company',
        yaxis_title='Score',
        barmode='stack',
        plot_bgcolor=colors['background'],
        paper_bgcolor=colors['background'],
        font=dict(color=colors['text']),
    )

    return fig

# Callback for the 3D scatter chart
@app.callback(
    Output('esg-3d-scatter-chart', 'figure'),
    Input('sector-dropdown', 'value')
)
def update_3d_scatter_graph(selected_sector):
    # Filter data based on selected sector
    filtered_df = df[df['Sector'] == selected_sector]

    # Ensure there is enough data to plot
    if filtered_df.empty:
        return go.Figure()

    # Create 3D scatter chart
    fig = px.scatter_3d(
        filtered_df,
        x='Environment Risk Score',
        y='Governance Risk Score',
        z='Social Risk Score',
        color='Total ESG Risk score',
        labels={
            'Environment Risk Score': 'Environment Risk Score',
            'Governance Risk Score': 'Governance Risk Score',
            'Social Risk Score': 'Social Risk Score',
            'Total ESG Risk score': 'Total ESG Risk Score'
        },
        title=f'3D Scatter Plot of ESG Scores for {selected_sector} Sector',
        hover_name='Name'  # 显示公司名称作为悬浮提示
    )

    fig.update_traces(marker=dict(size=5))  # 调整标记大小
    fig.update_layout(
        plot_bgcolor=colors['background'],
        paper_bgcolor=colors['background'],
        font=dict(color=colors['text']),
    )

    return fig

# 其他分析图的回调函数
# 示例：添加极值分析的柱状图回调
@app.callback(
    Output('esg-extreme-value-chart', 'figure'),
    Input('sector-dropdown', 'value')
)
def update_extreme_value_chart(selected_sector):
    # 过滤数据
    filtered_df = df[df['Sector'] == selected_sector]

    # 计算极值
    extreme_values = filtered_df[['Name', 'Total ESG Risk score']].copy()
    extreme_values['is_extreme'] = extreme_values['Total ESG Risk score'].abs() > extreme_values['Total ESG Risk score'].mean() + 2 * extreme_values['Total ESG Risk score'].std()

    # 创建极值柱状图
    fig = go.Figure(data=[go.Bar(
        x=extreme_values['Name'][extreme_values['is_extreme']],
        y=extreme_values['Total ESG Risk score'][extreme_values['is_extreme']],
        marker_color=colors['lemon_yellow'],
    )])

    # 更新布局
    fig.update_layout(
        title=f'Extreme ESG Scores for {selected_sector} Sector',
        xaxis_title='Company',
        yaxis_title='Total ESG Risk Score',
        plot_bgcolor=colors['background'],
        paper_bgcolor=colors['background'],
        font=dict(color=colors['text']),
    )

    return fig

In [5]:
if __name__ == '__main__':
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/
